In [1]:
from typing import Union, Iterable

import numpy
import plotly.graph_objects as go

In [5]:
Float = numpy.float32
Double = numpy.float64


def inverse_exponent(x: Union[Float, Double], steps: int = 1000) -> Union[Float, Double]:
    if not isinstance(x, (Float, Double)):
        raise ValueError(f"Unsupported type for x: {type(x)}.")
    x_type = type(x)
    result = term = x_type(1)
    for k in range(1, steps):
        term = term / x_type(k) * x
        result += term
    return x_type(1) / result

def get_relative_error(true_values: Iterable, calculated_values: Iterable) -> Iterable:
    return [
        Double(abs(_t - _c) / _t) for _t, _c in zip(true_values, calculated_values)
    ]

In [6]:
x_range = numpy.arange(100)
numpy_values = numpy.exp(-x_range.astype(numpy.float128))

In [7]:
float_values = [inverse_exponent(Float(i)) for i in x_range]
float_relative_errors = get_relative_error(numpy_values, float_values)
fig = go.Figure()
fig.add_trace(go.Scatter(x=-x_range, y=float_relative_errors, mode='lines+markers'))
fig.update_layout(
    yaxis={"title": "Log error", "exponentformat": "e"}, xaxis={"title": "x"}, title="Float relative error"
)
fig.update_yaxes(type="log")
fig.show()

<ipython-input-5-61e6b81acdbc>:12: RuntimeWarning:

overflow encountered in float_scalars

<ipython-input-5-61e6b81acdbc>:11: RuntimeWarning:

overflow encountered in float_scalars



In [5]:
double_values = [inverse_exponent(Double(i)) for i in x_range]
double_relative_errors = get_relative_error(numpy_values, double_values)
fig = go.Figure()
fig.add_trace(go.Scatter(x=-x_range, y=double_relative_errors, mode='lines+markers'))
fig.update_layout(
    yaxis={"title": "Log error", "exponentformat": "e"}, xaxis={"title": "x"}, title="Double relative error"
)
fig.update_yaxes(type="log")
fig.show()

In [6]:
def get_errors_by_n_steps(x: Union[Float, Double], n_steps: int) -> Iterable:
    true_values = numpy.exp(numpy.float128(-x)).repeat(n_steps)
    calc_values = [inverse_exponent(x, i) for i in range(n_steps)]
    return get_relative_error(true_values, calc_values)

n_teylor_steps = 100
float_re_10 = get_errors_by_n_steps(Float(10), n_teylor_steps)
float_re_20 = get_errors_by_n_steps(Float(20), n_teylor_steps)
double_re_10 = get_errors_by_n_steps(Double(10), n_teylor_steps)
double_re_20 = get_errors_by_n_steps(Double(20), n_teylor_steps)

x_range = [i for i in range(n_teylor_steps)]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_range, y=float_re_10, mode='lines+markers', name="Float relative error (x=-10)"))
fig.add_trace(go.Scatter(x=x_range, y=float_re_20, mode='lines+markers', name="Float relative error (x=-20)"))
fig.add_trace(go.Scatter(x=x_range, y=double_re_10, mode='lines+markers', name="Double relative error (x=-10)"))
fig.add_trace(go.Scatter(x=x_range, y=double_re_20, mode='lines+markers', name="Double relative error (x=-20)"))
fig.update_layout(yaxis={"title": "Log error", "exponentformat": "e"}, title="Error by number of steps")
fig.update_yaxes(type="log")
fig.show()

# Выводы

1. Начиная с определённого шага слагаемые ряда Тейлора становятся меньше относительной погрешности.
Причём чем выше точность, тем больше шагов Тейлора имеет смысл делать.
2. Значение `x` также влияет на необходимое число шагов. Чем выше абсолютное значение, тем больше шагов надо сделать.
2. Для чисел с одинарной точностью относительная ошибка находится в районе 10^(-7), а для двойной точности
в районе 10^(-16).
Таким образом, при увеличении точности в два раза, относительная погрешность уменьшается на 9 порядков.